# Inference and Reasoning with Bayesian Networks

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Assignment 2

Name: Yang Lu

Student ID: u6274652

## Instructions

|             |Notes|
|:------------|:--|
|Maximum marks| 19|
|Weight|19% of final grade|
|Format| Complete this ipython notebook. Do not forget to fill in your name and student ID above|
|Submission mode| Use [wattle](https://wattle.anu.edu.au/)|
|Formulas| All formulas which you derive need to be explained unless you use very common mathematical facts. Picture yourself as explaining your arguments to somebody who is just learning about your assignment. With other words, do not assume that the person marking your assignment knows all the background and therefore you can just write down the formulas without any explanation. It is your task to convince the reader that you know what you are doing when you derive an argument. Typeset all formulas in $\LaTeX$.|
| Code quality | Python code should be well structured, use meaningful identifiers for variables and subroutines, and provide sufficient comments. Please refer to the examples given in the tutorials. |
| Code efficiency | An efficient implementation of an algorithm uses fast subroutines provided by the language or additional libraries. For the purpose of implementing Machine Learning algorithms in this course, that means using the appropriate data structures provided by Python and in numpy/scipy (e.g. Linear Algebra and random generators). |
| Cooperation | All assignments must be done individually. Cheating and plagiarism will be dealt with in accordance with University procedures (please see the ANU policies on [Academic Honesty and Plagiarism](http://academichonesty.anu.edu.au)). Hence, for example, code for programming assignments must not be developed in groups, nor should code be shared. You are encouraged to broadly discuss ideas, approaches and techniques with a few other students, but not at a level of detail where specific solutions or implementation issues are described by anyone. If you choose to consult with other students, you will include the names of your discussion partners for each solution. If you have any questions on this, please ask the lecturer before you act. |

$\newcommand{\dotprod}[2]{\left\langle #1, #2 \right\rangle}$
$\newcommand{\onevec}{\mathbb{1}}$

Setting up the environment (there is some hidden latex which needs to be run in this cell).

In [1]:
import itertools, copy
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Image

%matplotlib inline

## Part 1: Graphical Models

### Problem setting

We are interested to predict the outcome of the election in an imaginary country, called Under Some Assumptions (USA). There are four candidates for whom the citizens can **Vote** for: Bernie, Donald, Hillary, and Ted. The citizens live in four **Region**s: north, south, east and west. We have general demographic information about the people, namely: **Gender** (male, female) and **Hand**edness (right, left). Based on surveys done by an external company, we believe that the **Region** and **Gender** affects whether the people use their **Jacket**s full time, part time or never. Surprisingly, the company told us that the **Age** of their shoes (new, worn, old) depends on how often they wear their **Jacket**s. Furthermore, the **Gender** and their preferred **Hand** affects the **Colour** of their hat (white, black). Finally, surveys say that the citizens will **Vote** based on their **Region**, **Age** of their shoes and **Colour** of their hats.

The directed graphical model is depicted below:

In [2]:
Image(url="https://machlearn.gitlab.io/isml2017/assignments/election_model.png")

### Conditional probability tables

After paying the survey firm some more money, they provided the following conditional probability tables.

|$p(R)$ | R=n | R=s | R=e | R=w |
|:-----:|:--:|:--:|:--:|:--:|
|marginal| 0.2 | 0.1 | 0.5 | 0.2 |

|$p(G)$ | G=m | G=f |
|:-----:|:--:|:--:|
|marginal| 0.3 | 0.7 |

|$p(H)$ | H=r | H=l |
|:-----:|:--:|:--:|
|marginal| 0.9 | 0.1 |

| $p(J|R,G)$ | R=n,G=m | R=n,G=f | R=s,G=m | R=s,G=f | R=e,G=m | R=e,G=f | R=w,G=m | R=w,G=f |
|:-----:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|**J**=full $\quad$  |0.9 |0.8 |0.1 | 0.3 |0.4 |0.01| 0.02 | 0.2  |
|**J**=part $\quad$  |0.08|0.17|0.03| 0.35|0.05|0.01| 0.2  | 0.08 |
|**J**=never $\quad$ |0.02|0.03|0.87| 0.35|0.55|0.98| 0.78 | 0.72 |

| $p(A|J)$ | J=full | J=part | J=never |
|:-----:|:--:|:--:|:--:|
|**A**=new  |0.01|0.96|0.3|
|**A**=worn |0.98|0.03|0.5|
|**A**=old  |0.01|0.01|0.2|

| $p(C|G,H)$ | G=m,H=r | G=m,H=l | G=f,H=r | G=f,H=l |
|:-----:|:--:|:--:|:--:|:--:|
|**C**=black $\quad$ |0.9 |0.83 |0.17 | 0.3 |
|**C**=white $\quad$ |0.1 |0.17|0.83 | 0.7 |

The final conditional probability table is given by the matrix below. The order of the rows and columns are also given below.

In [3]:
vote_column_names = ['north,new,black', 'north,new,white', 'north,worn,black', 'north,worn,white', 
                'north,old,black', 'north,old,white', 'south,new,black', 'south,new,white', 
                'south,worn,black', 'south,worn,white', 'south,old,black', 'south,old,white', 
                'east,new,black', 'east,new,white', 'east,worn,black', 'east,worn,white', 
                'east,old,black', 'east,old,white', 'west,new,black', 'west,new,white', 
                'west,worn,black', 'west,worn,white', 'west,old,black', 'west,old,white']

vote_outcomes = ('bernie','donald','hillary','ted')

vote_pmf_array = np.array(
        [
            [0.1,0.1,0.4,0.02,0.2,0.1,0.1,0.04,0.2,0.1,0.1 ,0.1,0.4 ,0.1 ,0.1,0.1 ,0.1,0.04,0.3,0.2,0.1,0.3,0.34,0.35],
            [0.3,0.4,0.2,0.5 ,0.1,0.2,0.1,0.5 ,0.1,0.2,0.5 ,0.3,0.2 ,0.42,0.2,0.67,0.4,0.4 ,0.1,0.1,0.5,0.1,0.1 ,0.1],
            [0.5,0.4,0.3,0.3 ,0.5,0.6,0.6,0.3 ,0.5,0.4,0.36,0.3,0.28,0.3 ,0.4,0.1 ,0.4,0.16,0.4,0.2,0.3,0.3,0.4 ,0.5],
            [0.1,0.1,0.1,0.18,0.2,0.1,0.2,0.16,0.2,0.3,0.04,0.3,0.12,0.18,0.3,0.13,0.1,0.4 ,0.2,0.5,0.1,0.3,0.16,0.05],
        ]
)

The 7 conditional probability tables in are encoded in python below. 

**Base your subsequent computations on these objects**.

In [4]:
class RandomVariable(object):
    def __init__(self, name, parents, outcomes, pmf_array):
        assert isinstance(name, str)
        assert all(isinstance(_, RandomVariable) for _ in parents)
        assert isinstance(outcomes, tuple)
        assert all(isinstance(_, str) for _ in outcomes)
        assert isinstance(parents, tuple)
        assert isinstance(pmf_array, np.ndarray)
        keys = tuple(map(tuple, itertools.product(*[_.outcomes for _ in parents])))
        assert np.allclose(np.sum(pmf_array, 0), 1)
        expected_shape = (len(outcomes), len(keys))
        assert pmf_array.shape == expected_shape, (pmf_array.shape, expected_shape)
        pmfs = {k: {outcome: probability for outcome, probability in zip(outcomes, probabilities)} 
                for k, probabilities in zip(keys, pmf_array.T)}
        self.name, self.parents, self.outcomes, self.pmfs = name, parents, outcomes, pmfs


class BayesianNetwork(object):
    def __init__(self, *random_variables):
        assert all(isinstance(_, RandomVariable) for _ in random_variables)
        self.random_variables = random_variables
        

region_pmf_array = np.array([[0.2, 0.1, 0.5, 0.2]]).T
region = RandomVariable(
    name='region',
    parents=tuple(),
    outcomes=('north', 'south', 'east', 'west'), 
    pmf_array = region_pmf_array,
)

gender_pmf_array = np.array([[0.3, 0.7]]).T
gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array
)

hand_pmf_array = np.array([[0.9, 0.1]]).T
hand = RandomVariable(
    name='hand',
    parents=tuple(),
    outcomes=('left', 'right'), 
    pmf_array = hand_pmf_array
)

jacket_pmf_array = np.array(
        [
            [0.9,0.8,0.1,0.3,0.4,0.01,0.02,0.2],
            [0.08,0.17,0.03,0.35,0.05,0.01,0.2,0.08],
            [0.02,0.03,0.87,0.35,0.55,0.98,0.78,0.72],
        ]
    )
jacket = RandomVariable(
    name='jacket',
    parents=(region, gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array
)

age_pmf_array = np.array(
        [
            [0.01,0.96,0.3],
            [0.98,0.03,0.5],
            [0.01,0.01,0.2],
        ]
    )
age = RandomVariable(
    name='age',
    parents=(jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array
)

colour_pmf_array = np.array(
        [
            [0.9,0.83,0.17,0.3],
            [0.1,0.17,0.83,0.7],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender, hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)


election_model = BayesianNetwork(region, gender, hand, jacket, age, colour, vote)

### 1A (1 mark) Joint distribution of a subset

- Compute the joint distribution of **Jacket**, **Region** and **Gender**. Print in a tab separated format the resulting distribution.

In [5]:
# Solution goes here
#Compute the joint using itertools to give all the combination of outcomes
for j, r, g in itertools.product(jacket.outcomes,region.outcomes,gender.outcomes):
    print('p({:^6},{:^6},{:^6})={:^2.2f}'.format(j,r,g,jacket.pmfs[(r,g)][j]*region.pmfs[()][r]*gender.pmfs[()][g]))
    #print('p(',j,r,g,')->',jacket.pmfs[(r,g)][j]*region.pmfs[()][r]*gender.pmfs[()][g])

p( full ,north , male )=0.05
p( full ,north ,female)=0.11
p( full ,south , male )=0.00
p( full ,south ,female)=0.02
p( full , east , male )=0.06
p( full , east ,female)=0.00
p( full , west , male )=0.00
p( full , west ,female)=0.03
p( part ,north , male )=0.00
p( part ,north ,female)=0.02
p( part ,south , male )=0.00
p( part ,south ,female)=0.02
p( part , east , male )=0.01
p( part , east ,female)=0.00
p( part , west , male )=0.01
p( part , west ,female)=0.01
p(never ,north , male )=0.00
p(never ,north ,female)=0.00
p(never ,south , male )=0.03
p(never ,south ,female)=0.02
p(never , east , male )=0.08
p(never , east ,female)=0.34
p(never , west , male )=0.05
p(never , west ,female)=0.10


### 1B1 (2 marks) Variable Ordering

1. Implement a function which determines an appropriate ordering of the variables for use in the following scheme:
    - For the first node R, draw a sample from p(R).
    - For each subsequent node, draw a sample from the conditional distribution $p(X \,|\, \text{parents}(X))$ where $\text{parents}(X)$ are the parents of the variable $X$ in the graphical model.
- Use your function to compute such an ordering and print the result in a human friendly format.

In [6]:
#recursively traverse all parent node
def check_parents(node,ordering):
    
    #if all parents in ordering, then add it to ordering 
    if set(node.parents) <= set(ordering) and node not in ordering:
        ordering.append(node)
    else:
        for parent in node.parents:check_parents(parent,ordering)
            
#Draw a order using recursive method to make their parents exist before given an order
def determine_order(model):
    ordering=[]
    unordered = set(list(model.random_variables))-set(ordering)
    while unordered !=set():
        check_parents(vote,ordering)
        #remove visited nodes
        unordered=set(model.random_variables)-set(ordering)
    return tuple(ordering)

print('The order is:')
for order,sample in enumerate(determine_order(election_model)):
    print(order,sample.name)


The order is:
0 region
1 gender
2 hand
3 jacket
4 colour
5 age
6 vote


### 1B2 (2 marks) Sampling

1. Given the ordering you have determined above, implement the sampler described in the previous question. If you were unable to compute the ordering in the previous question, use ``ordering = (hand, region, gender, colour, jacket, age, vote)``.
2. Draw a single sample and print the result in a human friendly format.

In [7]:
# Solution goes here
#Using np.random.choice function to draw a sample based on its probility
def sample_node(node,observed):
    condition = []
    for p in node.parents:
        for n in observed:
            if n in p.outcomes:
                condition.append(n)
    node_sample = np.random.choice(node.outcomes,p = [ node.pmfs[tuple(condition)][o] for o in node.outcomes])
    observed.append(node_sample)
    return node_sample

#Draw all the R,J,H,G,A,C,V sample once a time
def sample_allnodes(ordering):
    observed = []
    result = {}
    observed = [sample_node(node,observed) for node in ordering ] 
    
    for index ,node in enumerate(ordering):
        result[node.name]=observed[index]
    return result



print('A sample of p(R) is:',sample_node(region,observed = []))

ordering = determine_order(election_model)

print('A single sample by ordering is:\n',sample_allnodes(ordering))



A sample of p(R) is: north
A single sample by ordering is:
 {'region': 'east', 'gender': 'female', 'hand': 'left', 'jacket': 'never', 'colour': 'white', 'age': 'old', 'vote': 'ted'}


### 1B3 (2 marks) Marginals

1. Calculate (and show in LaTeX) the marginal distribution for **Jacket**.
- Implement a function which computes the marginal distribution of each variable. It should make use of the ordering used by your sampler.
- Implement a function which takes a list of samples the model and computes the empirical marginal distribution of each variable.
- Plot the theoretical and approximate marginals which you obtain along with the absolute percent error between the two, in the format below:

####  1. Calculate (and show in LaTeX) the marginal distribution for **Jacket**.

$p(J)=\sum\limits_{R,G,H,A,C}p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(C\,|\,G,H)p(V\,|\,R,A,C)$

####  2. Implement a function which computes the marginal distribution of each variable. It should make use of the ordering used by your sampler.

In [8]:
# Solution goes here
#Compute a node's probility 
def compute_node_probility(node,instance):
    #if this node donot have parent node
    if node.parents == ():
        outcome = list(set(node.outcomes) & set(instance))
        return node.pmfs[()][outcome[0]]
    #have parents, compute conditional probability
    for condition in itertools.product(*[parent.outcomes for parent in node.parents]):
        #Check which one we want to compute 
        if (set(condition) & set(instance) == set(condition)):
            outcome = list(set(node.outcomes) & set(instance))[0]
            product = node.pmfs[condition][outcome]
            return product

#Compute one node maginal distribution e.g. p(G) or p(V) or P(C)
def compute_onenode_marginal(ordering,node):
    #Initialize the node we want to compute
    result={name:0 for name in node.outcomes}
    #For all joints we comopute there joint distribution and sum up it 
    for instance in itertools.product(*[n.outcomes for n in ordering]):
        nodes_product = 1
        for nodes in ordering:
            nodes_product *= compute_node_probility(nodes,instance)
        #Check which one we want to sum
        out = list(set(node.outcomes) & set(instance))[0]
        result[out] += nodes_product
    return result

#Compute several nodes' joint distribution e.g. p(G,H) or p(G,V,H)
def compute_multinodes_marginal(ordering,nodes):
    #Initialize the node we want to compute
    result={joint:0 for joint in itertools.product(*[n.outcomes for n in nodes]) }
    flatten = lambda *n: (e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,))) 
    joint_outcomes = list(flatten([n.outcomes for n in nodes]))
    
    #For all joints we comopute there joint distribution and sum up it 
    for instance in itertools.product(*[n.outcomes for n in ordering]):
        node_product = 1
        for node in ordering:
            node_product *= compute_node_probility(node,instance)
        #Only sum up the one we interested in
        out = set(instance) & set(joint_outcomes)
        for r in result:
            if set(r)==out:
                result[r] += node_product   
    return result



#### 3.Implement a function which takes a list of samples the model and computes the empirical marginal distribution of each variable.

In [9]:
# Solution goes here
#Generate 100000 sample by sampler
def compute_empirical_distribution(ordering):
    samples = []
    for i in range(10000):
        samples.append(sample_allnodes(ordering))
        
    empirical_distribution={n.name: {x:0 for x in n.outcomes} for n in ordering}

    for node_name in empirical_distribution:
        for node_outcome in empirical_distribution[node_name]:    
            count = 0
            for sample in samples:
                if sample[node_name] == node_outcome:
                    count+=1
            empirical_distribution[node_name][node_outcome] = count/len(samples)
   
    return empirical_distribution


#### 4. Plot the theoretical and approximate marginals which you obtain along with the absolute percent error between the two

In [10]:
#Compute order of election_model
ordering = determine_order(election_model)
#Compute the empirical_distribution
empirical_marginal=compute_empirical_distribution(ordering)
marginal={}
#Compute the marginal distribution 
for node in ordering:
    marginal[node.name] = compute_onenode_marginal(ordering,node)

#Print in tables
def print_table(marginal, empirical_marginal):
    for m,em in zip(marginal,empirical_marginal):
        print(m)
        print('{:^7}{:^8}{:^8}{:^8}'.format('outcome','exact','approx','error'))
        for pm,pem in zip(marginal[m],empirical_marginal[em]):
            print('{:7}{:^8.2f}{:^8.2f}{:^8.2f}'.format(pm,marginal[m][pm],empirical_marginal[em][pem],abs(marginal[m][pm]-empirical_marginal[em][pem])/marginal[m][pm]*100))
        print('\n')
    print()
    
print_table(marginal,empirical_marginal)

region
outcome exact   approx  error  
north    0.20    0.20    1.05  
south    0.10    0.10    1.00  
east     0.50    0.50    0.12  
west     0.20    0.20    1.85  


gender
outcome exact   approx  error  
male     0.30    0.29    1.87  
female   0.70    0.71    0.80  


hand
outcome exact   approx  error  
left     0.90    0.90    0.33  
right    0.10    0.10    3.00  


jacket
outcome exact   approx  error  
full     0.28    0.28    0.50  
part     0.09    0.08    5.22  
never    0.63    0.63    0.51  


colour
outcome exact   approx  error  
black    0.40    0.39    0.83  
white    0.60    0.61    0.55  


age
outcome exact   approx  error  
new      0.28    0.27    1.89  
worn     0.59    0.60    1.14  
old      0.13    0.13    1.18  


vote
outcome exact   approx  error  
bernie   0.15    0.16    0.96  
donald   0.35    0.34    0.82  
hillary  0.30    0.29    1.52  
ted      0.20    0.21    2.86  





### 1B4 (1 mark) Easy conditional probabilities

Compute $p(X \,|\, G=\text{female})$ for all $X$ other than $G$,
1. Approximately, using your sampler
- Exactly, using your marginal calculating function from the previous question. Hint: what happens if you set  $p(G=\text{female})=1$ in your model?
- Plot the results side by side in the same format as the previous question.
- State for which variables other than $G$ the theoretical scheme above can be used to compute such conditionals, and why.

#### 1.Approximately, using your sampler

In [11]:
# Solution goes here
#Compute a node's empirical probility
def compute_empirical_X_givenG(nodeX):
    samples = []
    for i in range(10000):
        samples.append(sample_allnodes(ordering))
    x = {node_outcome : 0 for node_outcome in nodeX.outcomes}
    g=0
    for sample in samples:
        if sample['gender'] == 'female':
            g+=1
            for node_outcome in x:

                if node_outcome == sample[nodeX.name]:
                    x[node_outcome]+=1
    return {(node_outcome,'female') : x[node_outcome]/g for node_outcome in x}

#Compute all nodes' empirical probility
def compute_empirical_all_X_givenG(ordering):
    result={}
    for nodeX in ordering:
        result[nodeX.name]=compute_empirical_X_givenG(nodeX)
    del result['gender']
    return result

#calculate the approximate result by sampler
empirical = compute_empirical_all_X_givenG(ordering)

#### 2. Exactly, using your marginal calculating function from the previous question. 

In [12]:
#Use hint, set p(female) =1 
gender_pmf_array = np.array([[0, 1]]).T
gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array
)
gender.pmf_array = np.array([[0, 1]]).T
jacket_pmf_array = np.array(
        [
            [0.9,0.8,0.1,0.3,0.4,0.01,0.02,0.2],
            [0.08,0.17,0.03,0.35,0.05,0.01,0.2,0.08],
            [0.02,0.03,0.87,0.35,0.55,0.98,0.78,0.72],
        ]
    )
jacket = RandomVariable(
    name='jacket',
    parents=(region, gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array
)

age_pmf_array = np.array(
        [
            [0.01,0.96,0.3],
            [0.98,0.03,0.5],
            [0.01,0.01,0.2],
        ]
    )
age = RandomVariable(
    name='age',
    parents=(jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array
)

colour_pmf_array = np.array(
        [
            [0.9,0.83,0.17,0.3],
            [0.1,0.17,0.83,0.7],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender, hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)

modified_election_model = BayesianNetwork(region, gender, hand, jacket, age, colour, vote)

In [13]:
#Comput joint distribution p(X,female)
import collections
ordering = determine_order(modified_election_model)
def compute_marginal_X_givenG(ordering):
    result = {}
    del_male_result = collections.defaultdict(dict)
    for node in ordering:
        if node.name == 'gender':continue
        result[node.name]=compute_multinodes_marginal(ordering,(node,gender))
    for i in result:
        for j in result[i]:
            if j[1] =='male':continue
            del_male_result[i][j]=result[i][j]
    return del_male_result

#Calculate p(X,female)
marginal = compute_marginal_X_givenG(ordering)  

In [14]:
#Resume the model
gender_pmf_array = np.array([[0.3, 0.7]]).T
gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array
)
gender.pmf_array = np.array([[0, 1]]).T
jacket_pmf_array = np.array(
        [
            [0.9,0.8,0.1,0.3,0.4,0.01,0.02,0.2],
            [0.08,0.17,0.03,0.35,0.05,0.01,0.2,0.08],
            [0.02,0.03,0.87,0.35,0.55,0.98,0.78,0.72],
        ]
    )
jacket = RandomVariable(
    name='jacket',
    parents=(region, gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array
)

age_pmf_array = np.array(
        [
            [0.01,0.96,0.3],
            [0.98,0.03,0.5],
            [0.01,0.01,0.2],
        ]
    )
age = RandomVariable(
    name='age',
    parents=(jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array
)

colour_pmf_array = np.array(
        [
            [0.9,0.83,0.17,0.3],
            [0.1,0.17,0.83,0.7],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender, hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)

election_model = BayesianNetwork(region, gender, hand, jacket, age, colour, vote)

In [15]:
#Print in tables
def print_table(marginal, empirical_marginal):
    for m,em in zip(marginal,empirical_marginal):
        print(m)
        print('{:^15}{:^8}{:^8}{:^8}'.format('outcome','exact','approx','error'))
        for pm,pem in zip(marginal[m],empirical_marginal[em]):
            print('{:^7}|{:^7}{:^8.2f}{:^8.2f}{:^8.2f}'.format(pm[0],pm[1],marginal[m][pm],empirical_marginal[em][pem],abs(marginal[m][pm]-empirical_marginal[em][pem])/marginal[m][pm]*100))
        print('\n')
    print()
    
print_table(marginal,empirical)

region
    outcome     exact   approx  error  
 north |female   0.20    0.20    1.27  
 south |female   0.10    0.10    3.51  
 east  |female   0.50    0.50    0.03  
 west  |female   0.20    0.20    0.56  


hand
    outcome     exact   approx  error  
 left  |female   0.90    0.90    0.18  
 right |female   0.10    0.10    1.62  


jacket
    outcome     exact   approx  error  
 full  |female   0.23    0.23    2.27  
 part  |female   0.09    0.09    5.26  
 never |female   0.67    0.68    0.09  


colour
    outcome     exact   approx  error  
 black |female   0.18    0.19    2.77  
 white |female   0.82    0.81    0.62  


age
    outcome     exact   approx  error  
  new  |female   0.29    0.30    1.36  
 worn  |female   0.57    0.57    0.76  
  old  |female   0.14    0.14    0.30  


vote
    outcome     exact   approx  error  
bernie |female   0.13    0.13    0.52  
donald |female   0.39    0.39    0.00  
hillary|female   0.27    0.27    0.25  
  ted  |female   0.21    0.21    0.

#### 4. State for which variables other than $G$ the theoretical scheme above can be used to compute such conditionals, and why.

> Region and Gender, because they are start node and not condition on other nodes, therefore we can set one of its outcome become 1 and others 0 to compute conditional distribution. 

### 1B5 (3 marks) General conditional probabilities

1. Write down the expression of the joint probability $p(R,G,H,J,A,C,V)$ in terms of the conditional probabilities in the graphical model.
- Derive $p(G = male \,|\, V = Donald)$ in terms of the conditional probabilities of the graphical model.
- Compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is).

### Solution description

#### 1. Write down the expression of the joint probability  p(R,G,H,J,A,C,V)in terms of the conditional probabilities in the graphical model.

  
$$p(R,G,H,J,A,C,V) = p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(C\,|\,G,H)p(V\,|\,R,A,C)$$

#### 2. Derive  p(G=male|V=Donald) in terms of the conditional probabilities of the graphical model.

$$p(G \,|\, V) = \frac{p(G,V )}{p(V)} $$

$$p(G \,|\, V) = \frac{\sum\limits_{R,H,J,A,C}p(R,G,H,J,A,C,V)}{\sum\limits_{R,G,H,J,A,C}p(R,G,H,J,A,C,V)} \tag{Sum Rule}$$






#### 3. Compute and display in a human friendly format the conditional distributions  p(G=g|V=v)

In [16]:
# Solution goes here
def compute_P_G_given_V(model):
    #declear result dict
    p_G_given_V={}
    #Using new model which is elminated H
    ordering=determine_order(model)
    #Use new ordering for computing
    p_G_V = compute_multinodes_marginal(ordering,(gender,vote))
    p_V = compute_onenode_marginal(ordering,vote)

    for j in p_G_V:
        p_G_given_V[j] =p_G_V[j]/p_V[j[1]]
    return p_G_given_V

p_G_given_V =compute_P_G_given_V(election_model)
print('p( G | V )')
for j in p_G_given_V:
    print('p({:^6}|{:^8})={:2.2f}'.format(j[0],j[1],p_G_given_V[j]))



p( G | V )
p( male | bernie )=0.40
p( male | donald )=0.21
p( male |hillary )=0.36
p( male |  ted   )=0.28
p(female| bernie )=0.60
p(female| donald )=0.79
p(female|hillary )=0.64
p(female|  ted   )=0.72


### 1B6 (2 marks) Variable elimination

Denote the graphical model consider thus far $\mathcal{M}$.

1. Derive $p(R,G,J,A,C,V)$ by marginalising over $H$ in $\mathcal{M}$. 
- Describe how the structure (connectivity) of the new graphical model (call it $\mathcal{M}_H$) over all variables other than $H$ changes after eliminating $H$ in this way.
- Describe which conditional(s) in the new graphical model differ from the original model.
- Encode the $\mathcal{M}_H$ in python similarly to $\mathcal{M}$. 

#### 1. Derive $p(R,G,J,A,C,V)$ by marginalising over $H$ in $\mathcal{M}$. 
$$p(R,G,H,J,A,C,V) = p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(C\,|\,G,H)p(V\,|\,R,A,C)$$

$$p(R,G,J,A,C,V) = \sum\limits_{H}p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(C\,|\,G,H)p(V\,|\,R,A,C)$$

$$p(R,G,J,A,C,V) = p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(V\,|\,R,A,C)\sum\limits_{H}p(H)p(C\,|\,G,H)$$


#### 2. Describe how the structure (connectivity) of the new graphical model 
> p(H) disappear, and Gender point to C directly

$$p(R,G,J,A,C,V) = p(R)p(G)p(H)p(J\,|\,R,G)p(A\,|\,J)p(V\,|\,R,A,C)p(C\,|G)$$

#### 3. Describe which conditional(s) in the new graphical model differ from the original model.
> The conditional probility of C is different because in the new model $\mathcal{M}_H$

$$p(C|G) = \sum\limits_{H}p(H)p(C\,|\,G,H) $$

In [17]:
def p_C_given_G(ordering):
    result = {(name):0 for name in itertools.product(*[colour.outcomes,gender.outcomes])}
    flatten = lambda *n: (e for a in n for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,))) 
    joint_outcomes = list(flatten([n.outcomes for n in (colour,gender)]))
  
    #Compute the p(C,G) = sum{H} of P(C,G,H)
    for instance in itertools.product(colour.outcomes, gender.outcomes, hand.outcomes):
        product = 1
        product *= compute_node_probility(hand,instance)*compute_node_probility(colour,instance)
        out = set(instance) & set(joint_outcomes)
        for r in result:
            if set(r)==out:
                result[r] += product
    return result


#Compute new P(C|G) instead of P(C|G,H)
p_C_givenG = p_C_given_G(ordering)
print('New P(C|G) is:')
for i in p_C_givenG:
    print('p({:^6}|{:^6})={}'.format(i[0],i[1],p_C_givenG[i]))

New P(C|G) is:
p(black | male )=0.893
p(black |female)=0.18300000000000002
p(white | male )=0.10700000000000001
p(white |female)=0.817


#### 4. Encode the $\mathcal{M}_H$ in python similarly to $\mathcal{M}$. 

In [18]:
#Construct new model, modify the differed nodes 
colour_new_pmf_array = np.array(
        [
            #New conditional probility compute in previous question
            [0.89,0.18],
            [0.11,0.82],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender,),
    outcomes=('black', 'white'),
    pmf_array = colour_new_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)

#New model M_H
new_election_model = BayesianNetwork(region, gender, jacket, age, colour, vote)

### 1B6 (3 marks) General estimation of conditional probabilities (revisited)

1. As you did earlier, compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is). This time however, do so using $\mathcal{M}_H$ rather than $\mathcal{M}$.
- Quantify the computational advantages of using $\mathcal{M}_H$ in this way rather than $\mathcal{M}$.
- Which variable (or variables) would be the best to eliminate in this way, in terms of the aforementioned computational advantages, and why?
- Pick a (best) variable from the previous question and call it $X$. Assuming we have eliminated $X$, which would be the "best" variable to subsequently eliminate in a similar fashion?

In [19]:
# Solution goes here
#Use previous new model to compute p(G|V)
p_G_given_V =compute_P_G_given_V(new_election_model)
print('p(G|V) is:')
for j in p_G_given_V:
    print('p({:^6}|{:^8})={:2.2f}'.format(j[0],j[1],p_G_given_V[j]))

p(G|V) is:
p( male | bernie )=0.40
p( male | donald )=0.21
p( male |hillary )=0.36
p( male |  ted   )=0.28
p(female| bernie )=0.60
p(female| donald )=0.79
p(female|hillary )=0.64
p(female|  ted   )=0.72


#### 2. Quantify the computational advantages of using $\mathcal{M}_H$
> Reduce computational complexity, we only keep the variable that we are interested in, reduce the number of variables

#### 3. Which variable (or variables) would be the best to eliminate in this way, in terms of the aforementioned computational advantages, and why?

> Beside Hand, the best choice is Age and Colour 

> Because we should eliminate variable with fewest neighbors and fewest fill-in edges
 
 ** Reference ** Variable Elimination http://users.cecs.anu.edu.au/~jinbo/pr/lss4.pdf pg.15

#### 4. Pick a (best) variable from the previous question and call it $X$. Assuming we have eliminated $X$, which would be the "best" variable to subsequently eliminate in a similar fashion?

> After we eliminate Age, then eliminate Colour 

## Part 2: Theory

### 2A (3 marks) Functions of random variables

$u$ and $v$ are independently sampled from the standard uniform distribution on the unit interval, $[0,1]$. 

If $s=(2u-1)^2+(2v-1)^2 \geq 1$ then $x$ is sampled from the standard normal distribution, $\mathcal{N}(0,1)$. Otherwise $x=(2u-1)\sqrt{-2 \log(s)/s}$.

How is $x$ distributed?

### Solution description

$$p(x) = \int^{s} \int^{u} \int^{v}  p(x,s,u,v)dsdudv$$
$$=\int^{s} \int^{u} \int^{v} p(x|s)p(s|u,v)p(u)p(v)dsdudv$$

> Since $p(u)=1$ $p(v)=1$

$$=\int^{u} \int^{v} p(x|s\geq 1) p(s\geq 1|u,v)dudv + \int^{u} \int^{v} p(x|s < 1) p(s< 1|u,v)dudv$$

> We have 

$$p(x|s\geq 1) = \frac{1}{\sqrt{2\pi}}e^{-x^2}$$

$$p(x|s < 1) = (2u-1)\sqrt{-2 \log(s)/s}$$

$$p(s\geq 1|u,v) = p((2u-1)^2+(2v-1)^2 \geq 1| u,v)$$
$$p(s< 1|u,v) = p((2u-1)^2+(2v-1)^2 < 1| u,v)$$

$$=\int^{1}_{0} \int^{1}_{0} \frac{1}{\sqrt{2\pi}}e^{-x^2} \cdot p((2u-1)^2+(2v-1)^2 \geq 1| u,v)dudv + \int^{1}_{0} \int^{1}_{0} (2u-1)\sqrt{-2 \log(s)/s} \cdot p((2u-1)^2+(2v-1)^2 < 1| u,v)dudv$$



$$=\frac{1}{\sqrt{2\pi}}e^{-x^2}\int^{1}_{0} \int^{1}_{0} p((u-\frac{1}{2})^2+(v-\frac{1}{2})^2 \geq \frac{1}{4}| u,v)dudv+\int^{1}_{0} \int^{1}_{0} (u-1)\sqrt{-2 \log(s)/s} \cdot p((u-\frac{1}{2})^2+(v-\frac{1}{2})^2 < \frac{1}{4}| u,v)dudv$$

$$\rho cos\theta = u-\frac{1}{2}$$
$$\rho sin\theta = v-\frac{1}{2}$$
$$x = cos\theta\sqrt{-4ln\rho}\tag{Substitution}$$
$$x^2 = -4cos^2\theta ln\rho$$
$$\rho = e^{\frac{-x^2}{4cos^2\theta}}$$

$$=\frac{1}{\sqrt{2\pi}}e^{-x^2}\cdot\frac{1}{4}\pi + \frac{1}{\sqrt{2\pi}}e^{-x^2}\cdot(1-\frac{1}{4}\pi) $$
$$=\frac{1}{\sqrt{2\pi}}e^{-x^2}$$
> So $x$ ~ $\mathcal{N}(0,1) $